## Mapping Scottish place name origins using LLMs

In [ ]:
import os
import pandas as pd
import folium
import seaborn as sns
from tqdm import tqdm 
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain_core.runnables import RunnableLambda


### Run LLM query

In [ ]:
# Enable progress bar
tqdm.pandas()

# Load environment variables
load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY")

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=openai_api_key)

# Define the Pydantic Model
class GaelicCheckResponse(BaseModel):
    origin: str = Field(..., description="Whether the place name has one of the following origins: Scottish Gaelic, Norse, Pict, Scots or English. Use only this list. If you are not sure say 'Not sure'.")
    reason: str = Field(..., description="Short explanation of your reason for selecting the place name origin.")

# Define Output Parser
parser = PydanticOutputParser(pydantic_object=GaelicCheckResponse)

# Define the Prompt
prompt = PromptTemplate(
    template="""
    You are an expert in the etymology and origin of place names.
    Determine the origin of the following place names.
    
    Place Name: {place_name}
    
    Respond in JSON format following this schema:
    {format_instructions}

    """,
    input_variables=["place_name"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Define the chain
origin_chain = (
    RunnableLambda(lambda name: {"place_name": name})  # Convert string input to dict
    | prompt  # Format the prompt
    | llm  # Call the LLM
    | parser  # Parse response into structured JSON
    | RunnableLambda(lambda response: response.model_dump()) # Convert Pydantic model to dict
  )  

def process_place_name(place_name):
    try:
        result = origin_chain.invoke(place_name)  # Call the LLM chain
        return result["origin"], result["reason"]  # Extract fields
    except Exception as e:
        return None, f"Error: {e}"  # Handle errors

df = pd.read_csv('data/IPN_GB_2024.csv', encoding='latin-1')
df = df[(df.ctry23nm == 'Scotland') & (df.descnm == 'LOC')]
df = df.sample(500)

df[['origin', 'reason']] = df['place23nm'].progress_apply(lambda x: pd.Series(process_place_name(x)))

# Save data
# df.to_csv('data/IPN_GB_2024_with_origin.csv', index=False)
# df[['origin', 'reason']]

In [ ]:
df[['origin', 'reason']]

### Check output

In [ ]:
# df.to_csv('data/test.csv')
df_out = pd.read_csv('data/IPN_GB_2024_with_origin.csv')
df_out[['origin', 'reason']]

In [ ]:
# Create a Folium Map centered at an average location
map_center = [df_out["lat"].mean(), df_out["long"].mean()]
m = folium.Map(location=map_center, zoom_start=5)

# Function to assign colors
def get_marker_color(language):
    return {
        "Scottish Gaelic": "green",
        "Norse": "red",
        "Scots": "blue",
        "English": "purple",
        "Pict": "pink"
    }.get(language, "gray")  # Default to gray if unknown

# Add Markers
for _, row in df_out.iterrows():
    folium.Marker(
        location=[row["lat"], row["long"]],
        popup=f"{row['place23nm']} - {row['reason']}",
        icon=folium.Icon(color=get_marker_color(row["origin"]))
    ).add_to(m)

m


In [ ]:
sns.histplot(data=df_out, x='origin', stat='percent')